# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
#!pip install --upgrade panel hvplot bokeh
!pip install cartopy

In [2]:
!pip install geoviews


In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import json
import bokeh.models as bm


#print(bm.Column)






# Import API key
from api_keys import geoapify_key

geoapify_key = "d5a811a0280148fbad47bda86e077f8e"

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,paoua,7.2427,16.4406,70.65,88,67,3.71,CF,1689804223
1,1,blackmans bay,-43.0167,147.3167,45.73,74,100,13.94,AU,1689804223
2,2,arawa,-6.2132,155.5367,81.37,77,75,4.27,PG,1689804223
3,3,puerto natales,-51.7236,-72.4875,32.20,92,100,2.26,CL,1689804223
4,4,la passe,45.5549,-0.8967,64.98,88,17,8.10,FR,1689804223


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

#ESRI satellite map
#tiles=True
#tiles=

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 88) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()
# Display sample data
ideal_cities_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,namibe,-15.1961,12.1522,61.93,87,0,4.21,AO,1689804225
26,26,hwange,-18.3693,26.5019,57.67,41,0,1.83,ZW,1689804225
44,44,abu al matamir,30.9102,30.1744,78.62,62,0,2.73,EG,1689804053
183,183,wabu,37.5897,127.2203,74.82,86,0,1.36,KR,1689804244
217,217,florence,43.7667,11.2500,80.28,64,0,1.14,IT,1689804000


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,namibe,AO,-15.1961,12.1522,87,
26,hwange,ZW,-18.3693,26.5019,41,
44,abu al matamir,EG,30.9102,30.1744,62,
183,wabu,KR,37.5897,127.2203,86,
217,florence,IT,43.7667,11.2500,64,
266,genhe,CN,50.7833,121.5167,94,
331,caconda,AO,-13.7333,15.0667,43,
334,tympaki,GR,35.0729,24.7685,49,
345,sabha,LY,27.0377,14.4283,22,
350,madalena,PT,41.2162,-8.3345,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #filter_param = {
        #"circle": lon,lat,radius
        #}
    
    
    # Convert the filter object to a JSON string
    #filter_param_str = json.dumps(filter_param)
    
    # Add the filter parameter to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
 #     Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    

# Display sample data
hotel_df

Starting hotel search
hwange - nearest hotel: No hotel found
abu al matamir - nearest hotel: No hotel found
genhe - nearest hotel: No hotel found
caconda - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
25,namibe,AO,-15.1961,12.1522,87,Hotel Chik Chik Namibe
26,hwange,ZW,-18.3693,26.5019,41,No hotel found
44,abu al matamir,EG,30.9102,30.1744,62,No hotel found
183,wabu,KR,37.5897,127.2203,86,동서울관광호텔
217,florence,IT,43.7667,11.2500,64,Old bridge Apartment
266,genhe,CN,50.7833,121.5167,94,No hotel found
331,caconda,AO,-13.7333,15.0667,43,No hotel found
334,tympaki,GR,35.0729,24.7685,49,Sifogiannis
345,sabha,LY,27.0377,14.4283,22,فندق الكاظم-سبها
350,madalena,PT,41.2162,-8.3345,87,Hotel de Paredes


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)